In [ ]:
!pip install --upgrade pip
!pip install tensorflow

!pip3 install tf_pose

!sudo pip install numpy --upgrade --ignore-installed

import numpy
numpy.version.version

!pip3 install numpy==1.15.1

In [ ]:
import numpy
numpy.version.version

In [4]:
import sys
import time
import cv2
import math
import logging
import numpy as np
import matplotlib.pyplot as plt


from tf_pose import common
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

In [8]:
logger = logging.getLogger('TfPoseEstimatorRun')
logger.handlers.clear()
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [ ]:
model='mobilenet_thin'
resize='432x368'
w, h = model_wh(resize)
if w == 0 or h == 0:
    e = TfPoseEstimator(get_graph_path(model), target_size=(432, 368))
else:
    e = TfPoseEstimator(get_graph_path(model), target_size=(w, h))

# Pose Estimation, Key point estimation

In [ ]:
# Reading and Plotting the image
image_path = 'Image2.png'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.grid();

In [ ]:
print('Image array:',image)
print('Image Shape:', image.shape)

In [51]:
def img_plt(image, axis=False, grid=False, showBG = True):
    plt.figure(figsize=(15,8))
    if showBG:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image);
    if grid == True:
        plt.grid();
    if axis == False:       
        plt.axis('off');

In [ ]:
#Using Function for showing the image
image_path = 'Image2.png'
img_plt(cv2.imread(image_path), axis=True, grid=True)

In [53]:
# Change image for further procassing
image = common.read_imgfile(image_path, None, None)

# Nose = 0
# Neck = 1
# RShoulder = 2
# RElbow = 3
# RWrist = 4
# LShoulder = 5
# LElbow = 6
# LWrist = 7
# RHip = 8
# RKnee = 9
# RAnkle = 10
# LHip = 11
# LKnee = 12
# LAnkle = 13
# REye = 14
# LEye = 15
# REar = 16
# LEar = 17
# Background = 18

In [ ]:
print('Image array:',image)
print('Image Shape:', image.shape)

In [ ]:
# e.inference method gives keypoints coordinates
humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=4.0)
print('Heatmat Shape:',e.heatMat.shape)
print('Heatmat:',e.heatMat)

print(humans)

In [ ]:
#Plotting Heatmap
max_prob = np.amax(e.heatMat[:, :, :-1], axis=2)
plt.imshow(max_prob)
plt.grid();

In [ ]:
#Heatmap superimposed on image
plt.figure(figsize=(15,8))
bgimg = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_BGR2RGB)
bgimg = cv2.resize(bgimg, (e.heatMat.shape[1], e.heatMat.shape[0]), interpolation=cv2.INTER_AREA)
plt.imshow(bgimg, alpha=0.5)
plt.imshow(max_prob, alpha=0.5)
plt.colorbar()
plt.grid();

In [ ]:
# Maximum probability pairs
print('PAF matrix shape',e.pafMat.shape) 

tmp2 = e.pafMat.transpose((2, 0, 1))
tmp2_odd = np.amax(np.absolute(tmp2[::2, :, :]), axis=0)
tmp2_even = np.amax(np.absolute(tmp2[1::2, :, :]), axis=0)

fig = plt.figure(figsize=(20,15))

a = fig.add_subplot(2, 2, 3)
a.set_title('Vectormap-x')
plt.imshow(tmp2_odd, alpha=0.5)
plt.colorbar()

a = fig.add_subplot(2, 2, 4)
a.set_title('Vectormap-y')
plt.imshow(tmp2_even, alpha=0.5)
plt.colorbar()
plt.show()

In [30]:
image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

In [ ]:
img_plt(image, axis=True, grid=True)

In [ ]:
#Plotting the skeleton
image = common.read_imgfile(image_path, None, None)
humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=4.0)
black_background = np.zeros(image.shape)
skeleton = TfPoseEstimator.draw_humans(black_background, humans, imgcopy=False)
plt.figure(figsize=(15,8))
plt.imshow(skeleton);
plt.grid();      
plt.axis('off');

##

In [ ]:
keypoints = str(str(str(humans[0]).split('BodyPart:')[1:]).split('-')).split(' score=')
len(keypoints)

In [ ]:
keypoints

In [ ]:
#NEW PART1
index_array = str(humans[0]).split('BodyPart:')[1:]

new_index_array=[]
for i in index_array:
   new_index_array.append(i.split('-')[0])

new_index_array = list(map(int, new_index_array))
new_index_array

In [39]:
# Keypoints list index-wise
keypoints_list=[]
j=0
for i in range (17):

  if (i==int(new_index_array[j])):
    pnt = keypoints[j][-11:-1]
    pnt = tuple(map(float, pnt.split(', ')))
    keypoints_list.append(pnt)
    j=j+1

  else:
    keypoints_list.append((0,0))  

print(keypoints_list)

In [ ]:
# Retrieving original key points coordinates
keypts_array = np.array(keypoints_list)
keypts_array = keypts_array*(image.shape[1],image.shape[0])
keypts_array = keypts_array.astype(int)
keypts_array

# Angle calculation using keypts_array (Kinematic Feature Extraction)

In [ ]:
# Angle (11,1,5) Left Trunk Angle
import numpy as np

a = keypts_array[11]
b = keypts_array[1]
c = keypts_array[5]

ba = a - b
bc = c - b

cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
angle = np.arccos(cosine_angle)
ang_deg = np.degrees(angle)

print(ang_deg)

In [ ]:
# Angle (2,1,8) Right Trunk Angle
import numpy as np

a = keypts_array[2]
b = keypts_array[1]
c = keypts_array[8]

ba = a - b
bc = c - b

cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
angle = np.arccos(cosine_angle)
ang_deg = np.degrees(angle)

print(ang_deg)

Distance bw ref point and rankle keypoint (#10) calculation for the frontal frame done

In [ ]:
import math

kpt10 = keypts_array[10]
refpt = np.array([2150, 2000])

dist = math.dist(kpt10, refpt)

print(dist)

In [ ]:
# Angle (8,9,10) Right Knee Angle - Lateral frame

import numpy as np

a = keypts_array[8]
b = keypts_array[9]
c = keypts_array[10]

ba = a - b
bc = c - b

cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
angle = np.arccos(cosine_angle)
ang_deg = np.degrees(angle)

print(ang_deg)

In [ ]:
# Angle (9,8,1) Right Hip Angle - Lateral frame

import numpy as np

a = keypts_array[9]
b = keypts_array[8]
c = keypts_array[1]

ba = a - b
bc = c - b

cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
angle = np.arccos(cosine_angle)
ang_deg = np.degrees(angle)

print(ang_deg)

Thresholding on above values help with the classification part

In [45]:
def human_pose(image_path, showBG = True):
    image = common.read_imgfile(image_path, None, None)
  
    if image is None:
        logger.error('Image can not be read, path=%s' % image)
        sys.exit(-1)

    t = time.time()
    
    humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=4.0)
    elapsed = time.time() - t

    if showBG == False:
        image = np.zeros(image.shape)
    image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
    return image, humans

In [46]:
def keypoints_plt(image, hum, human=1, color='orange', showBG = True):
    if human == 0: human = 1
    num_hum = len(hum)
    keypoints = str(str(str(hum[human-1]).split('BodyPart:')[1:]).split('-')).split(' score=')
    keypoints_list=[]
    for i in range (len(keypoints)-1): 
        pnt = keypoints[i][-11:-1]
        pnt = tuple(map(float, pnt.split(', ')))
        keypoints_list.append(pnt)

    keypts_array = np.array(keypoints_list)
    keypts_array = keypts_array*(image.shape[1],image.shape[0])
    keypts_array = keypts_array.astype(int)
    keypts_array

    plt.figure(figsize=(10,10))
    plt.axis([0, image.shape[1], 0, image.shape[0]])  
    plt.scatter(*zip(*keypts_array), s=200, color=color, alpha=0.6)
    if showBG:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)      
    plt.imshow(image)
    ax=plt.gca() 
    ax.set_ylim(ax.get_ylim()[::-1]) 
    ax.xaxis.tick_top() 
    plt.title('Keypoints Person [{}] from {} humans detected\n'.format(human, num_hum))
    plt.grid();

    for i, txt in enumerate(keypts_array):
        ax.annotate(i, (keypts_array[i][0]-5, keypts_array[i][1]+5))
            
    return keypts_array

In [ ]:
image_path = 'Image2.png'
img, hum = human_pose(image_path, showBG = True)
img_plt(img, axis=True, grid=True)

hum

In [ ]:
image_path = 'Image2.png'
img, hum = human_pose(image_path, showBG=False)
keypoints = keypoints_plt(img, hum, showBG=False)

In [ ]:
# Starting code for frame-wise analysis
import cv2
  
def FrameCapture0(path):
  
    vidObj = cv2.VideoCapture(path)
    count = 0
    success = 1
  
    while success:
        success, image = vidObj.read()
        cv2.imwrite("/content/sample_data/s3/frame%d.jpg" % count, image)
        count += 1

In [ ]:
!zip -r /content/sample_data/s3.zip /content/sample_data/s3